# Woolsey Fire Data Exploration

This notebook plots satellite and model data related to the 2018 Woolsey Fire, including vegetation, climate, topography, and fire history. The Woolsey Fire ignited on **8 Nov 2018** and was fully contained on **21 Nov 2018**.

In [ ]:
import os

import earthpy as et
import earthpy.plot as ep
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xarray as xr

from ea_drought_burn.config import DATA_DIR
from ea_drought_burn.notebooks import run_notebook
from ea_drought_burn.utils import draw_legend, plot_bands, plot_rgb




# Set working directory to the earthpy data directory
os.chdir(os.path.join(DATA_DIR, "woolsey-fire"))

# Set base plot style
sns.set(font_scale=1.2, style="white")
plt.rc("figure.constrained_layout", use=True, h_pad=15/72, w_pad=15/72)

# Disable interpolation to prevent NaN erosion
plt.rc("image", interpolation=None)

In [ ]:
def plot_many(
    bands,
    titles,
    boundary,
    width=14,
    suptitle=None,
    ncols=2,
    **kwargs
):
    """Plots any number of bands in two columns
    
    Arguments
    ---------
    bands: list of arrays
        arrays to plot
    titles: list of str
        titles for each axis
    boundary: geopandas.GeoDataFrame
        boundary to draw on plot
    width: int or float (optional)
        width of the figure in inches
    suptitle: str (optional)
        title for the figure
    ncols: int
        number of columns
    kwargs:
        any keyword argument accepted by `plot_bands`
        
    Returns
    -------
    None
    """
    
    # Scale fonts
    sns.set(font_scale=1.2 * width / 14, style="white")
    
    # Scale outline
    bound_kwargs = {"edgecolor": "black", "facecolor": "none", "linewidth": 2}
    bound_kwargs["linewidth"] *= width // 14
    
    # Set height based on number rows
    nrows = int(np.ceil(len(bands) / ncols))
    height = (width / ncols - 1.5) * nrows

    fig, axes = plt.subplots(nrows, ncols, figsize=(width, height))
    
    if suptitle:
        fig.suptitle(suptitle)
    
    # Flatten axes into a 1D list
    axes_flat = []
    for row in axes:
        try:
            axes_flat.extend(row)
        except TypeError:
            axes_flat.append(row)
          
    # Delete last axis if not an even number of bands
    if len(bands) % 2:
        axes_flat[-1].remove()

    for ax, title, band in zip(axes_flat, titles, bands):
        plot_bands(band, ax=ax, title=str(title), **kwargs)
        boundary.plot(ax=ax, **bound_kwargs)
    
    sns.set(font_scale=1.2, style="white")

## Restore and prepare data

To ensure consitency across notebooks and avoid repeating code, all data is loaded using **load_data.ipynb** and restored as needed in individual notebooks. Please see that notebook for details about the sources of data used here.

In [ ]:
# Check for stored ready variable and run load-data.ipynb if not found
%store -r woolsey_data_ready
try:
    woolsey_data_ready
except NameError:
    print("Running load-data.ipynb...")
    run_notebook("load-data.ipynb")

# Retore variables using storemagic. Each variable is restored explicitly to
# avoid confusion about where variable names are coming from.
%store -r all_data
%store -r reproj_to
%store -r res
%store -r prism_2018

%store -r hill_fire
%store -r woolsey_fire
%store -r hill_and_woolsey_fires

%store -r cmap_dnbr
%store -r labels_dnbr
%store -r cmap_vegetation
%store -r labels_vegetation
%store -r prism_grid

# Set crop and fire boundaries
fire_name = "Woolsey Fire"
fire_bound = hill_and_woolsey_fires
crop_bound = fire_bound.envelope

# Crop all data to the Hill Fire if using that as bound
if fire_bound.iloc[0].geometry == hill_fire.iloc[0].geometry:
    all_data = {k: v.rio.clip(crop_bound) for  k, v in all_data.items()}

## SRTM topographic data

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 11))
fig.suptitle(f"{fire_name} - Topography - SRTM - {res} m")
plot_bands(all_data["Elevation"], ax=ax1, title="Elevation")
plot_bands(all_data["Slope"], ax=ax2, title="Slope")
plot_bands(all_data["Aspect"], ax=ax3, title="Aspect")
plot_bands(all_data["Folded Aspect"], ax=ax4, title="Folded Aspect")

# Plot Woolsey Fire boundary
for ax in (ax1, ax2):
    fire_bound.plot(ax=ax, edgecolor="white", facecolor="none", linewidth=2)
for ax in (ax3, ax4):
    fire_bound.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=2)

**Description:** Topographic data for the Woolsey Fire scar, including elevation, slope, aspect, and folded aspect. Folded aspect. calculated as *|180 - |aspect - 225||*, converts aspect into a linear parameter where hotness/dryness increases moving from 0° (for a northeast-facing slope) to 180° (for a southwest-facing slope) (McCune and Keon, 2002).

## Sentinel-2 pre- and post-fire images

Pre- and post-fire Sentinel-2 imagery is much better than the available Landsat images, which are obscured by smoke or clouds through early 2019.

In [ ]:
# Set size and style
sns.set(font_scale=1.2, style="white")

# Plot the pre- and post-fire images
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
fig.suptitle(f"{fire_name} - Sentinel-2 imagery - {res} m")

# Plot pre-fire image
plot_rgb(all_data["Sentinel-2 Prefire"], 
         ax=ax1,
         rgb=[3, 2, 1],
         stretch=True, title="Pre-fire - 31 Oct 2018")

# Plot post-fire image
plot_rgb(all_data["Sentinel-2 Postfire"],
         ax=ax2,
         rgb=[3, 2, 1],
         stretch=True,
         title="Post-fire - 15 Dec 2018")

# Plot Woolsey Fire boundary
for ax in (ax1, ax2, ax3, ax4):
    fire_bound.plot(ax=ax, edgecolor="white", facecolor="none", linewidth=2)

### Spectral indices

I calculated the following spectral indices based on the Sentinel-2 data:

+ **[Normalized Difference Vegetation Index (NDVI)](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-normalized-difference-vegetation-index)**
  assesses vegetation health. Calculated as `(nir_842 - red) / (nir_842 + red)`.
+ **[Soil Adjusted Vegetation Index (SAVI)](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-soil-adjusted-vegetation-index)**
  assesses vegetation health modified for low-cover pixels. Calculated as 
  `1.5 * (nir_842 - red) / (nir_842 + red + 0.5)`
+ **Normalized Difference Water Index (NDWI)** estimates vegetation water content.
  Formula modified from Gao (1996) to use an the 1610 nm band instead of the 1240
  nm band. Calculated as
  `(nir_865 - swir_1610) / (nir_865 + swir_1610)`.
+ **[Normalized Difference Moisture Index (NDMI)](https://www.usgs.gov/core-science-systems/nli/landsat/normalized-difference-moisture-index)**
  estimates vegetation water content. Calculated as
  `(nir_842 - swir_1610) / (nir_842 + swir_1610)`.
  
NDVI and SAVI are similar, and NDWI and NDMI are *very* similar.

In [ ]:
# Set size and style
sns.set(font_scale=1.2, style="white")

# Plot pre-fire Sentinel-2 spectral indides
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 11))
fig.suptitle(f"{fire_name} - Sentinel-2 Spectral Indices - {res} m\n"
              "31 Oct 2018 (pre-fire)")

axes = (ax1, ax2, ax3, ax4)
keys = ("NDVI (2018)", "SAVI (2018)", "NDWI (2018)", "NDMI (2018)")

for key, ax in zip(keys, axes):
    plot_bands(all_data[key],
               ax=ax,
               cmap="RdYlGn",
               vmin=-1,
               vmax=1,
               title=key)

# Plot Woolsey Fire boundary
for ax in (ax1, ax2, ax3, ax4):
    fire_bound.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1)

**Description:** Pre-fire NDMI and NDWI show a pattern broadly similar to the one seen in the FAL data within the Woolsey Fire scar. Low NDVI values (below 0 in much of the scar) indicate some combination of low moisture content and low cover.

### Burn severity

In [ ]:
# Set size and style
sns.set(font_scale=1.2, style="white")

# Plot Sentinel-2 dNBR and classified dNBR
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5.5))

plot_bands(all_data["Sentinel-2 dNBR"],
           ax=ax1,
           cmap="RdYlGn_r",
           vmin=all_data["MTBS dNBR"].min(),  # use MTBS min so plots same
           vmax=all_data["MTBS dNBR"].max(),  # use MTBS max so plots same
           title=(f"{fire_name} - Sentinel dNBR - {res} m\n"
                   "31 Oct 2018 - 15 Dec 2018"))

plot_bands(all_data["Sentinel-2 Classified dNBR"],
           ax=ax2,
           cmap=cmap_dnbr,
           vmin=1,
           vmax=5,
           title=(f"{fire_name} - Sentinel-2 classified dNBR - {res} m\n"
                   "31 Oct 2018 - 15 Dec 2018"),
           cbar=False,
           scale=False)

# Draw legend
draw_legend(im_ax=ax2.get_images()[0],
            classes=range(5),
            titles=labels_dnbr,
            bbox=(0.01, 0.99))

# Plot Woolsey Fire boundary
for ax in (ax1, ax2):
    fire_bound.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1)

**Description:** The Sentinel-2 burn-severity plots show a similar pattern to the MTBS plots below but have generally higher burn severities, including a larger number of high-severity pixels and a smaller number of unburned pixels.

## MTBS burn severity plots

MTBS provides normalized, field-validated dNBR plots for recent U.S. wildfires that are more reliable than those that, like the one I calculated above, are made solely from satellite images. A key benefit of the MTBS data is that field validation allows the mappers to use fire-specific bins for assessing the classes of burn severity.

In [ ]:
# Set size and style
sns.set(font_scale=1.2, style="white")

# Plot MTBS dNBR and classified dNBR
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5.5))

plot_bands(all_data["MTBS dNBR"],
           ax=ax1,
           cmap="RdYlGn_r",
           title=(f"{fire_name} - MTBS dNBR - {res} m\n"
                   "Dec 15, 2017 - Dec 15, 2018"))

plot_bands(all_data["MTBS Classified dNBR"],
           ax=ax2,
           cmap=cmap_dnbr,
           vmin=1,
           vmax=5,
           title=(f"{fire_name} - MTBS classified dNBR - {res} m\n"
                   "Dec 15, 2017 - Dec 15, 2018"),
           cbar=False,
           scale=False)

# Draw legend
draw_legend(im_ax=ax2.get_images()[0],
            classes=range(5),
            titles=labels_dnbr,
            bbox=(0.01, 0.99))


# Plot Woolsey Fire boundary
for ax in (ax1, ax2):
    fire_bound.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1)

## Woolsey Fire Stack

The original data included a second stack, WoolseyFireDataStack2, that included some Landsat data and BARC burn severity data, as well as the FAL data included in the SMM stack. All of that content is replicated elsewhere, and this file is not included in the final version of the data for this project.

+ **Landsat:** The WoolseyFireDataStack2 file contains two sets of Landsat files,
  one from 2017-10-24 and one from 2018-10-18. Both pre-date the Woolsey fire and
  include only bands 4 (red) and 7 (SWIR2). None of the spectral indices I'm aware
  of use those two bands.
+ **BARC:** BARC stands for Burned Area Reflectance Classification and is a 
  burn-severity index similar to dNBR. We decided to use the MTBS dNBR data
  instead.

## Santa Monica Mountain Stack

The Santa Monica Mountain (SMM) stack includes data about the vegetation and climate of the Santa Monica Mountain region in 2013-2016, which was a period of extreme to exceptional drought in southern California. This data was provided by Natasha Stavros and was compiled for a study about the drought (Dagit et al., 2017).

### Vegetation community

Vegetation communities as estimated for the Dagit et al. (2017) study are plotted below.

In [ ]:
# Set size and style
sns.set(font_scale=1.2, style="white")

# Plot vegetation community
fig, ax = plt.subplots(figsize=(12, 7))
plot_bands(all_data["Community"],
           ax=ax,
           title="Santa Monica Mountains vegetation communities - 30 m",
           cmap=cmap_vegetation,
           cbar=False)
fire_bound.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=2)

# Calculate percentages for each community
total = np.sum(np.isfinite(all_data["Community"]))
titles = []
for i, label in enumerate(labels_vegetation):
    pct = (np.sum(all_data["Community"] == i + 1) / total).item()
    titles.append(f"{label} ({pct:.0%})")

# Draw legend
ep.draw_legend(ax.get_images()[0],
               bbox=(1.02, 1),
               titles=titles,
               classes=range(len(titles)))

pass

**Description:** Many of the salients into and low-severity burn areas within the fire scar are classified as substrate. Moderate- and high-severity burns appear mostly in the chapparal-dominated southern half of the scar.

In [ ]:
# Set size and style
sns.set(font_scale=1.2, style="white")

bands = []
for i in range(1, 7):
    bands.append(all_data["Community"].where(all_data["Community"] == i) \
                                      .rio.clip(fire_bound.geometry))

# Plot each vegetation community separately
plot_many(bands,
          titles,
          fire_bound,
          suptitle="Woolsey Fire scar vegetation communities",
          cmap="Greys_r",
          cbar=False,
          scale=False)

**Description:** The main thing to note here is that the distribution of oak woodlands closely matches the burn-severity pattern in the MTBS data.

In [ ]:
# Set size and style
sns.set(font_scale=1.3, style="white")

# Plot burn-severity histograms for each community
dnbr_colors = [
    "#006400",
    "#7FFFD4",
    "#FFFF00",
    "#FF0000",
    "#7FFF00"
]

comm = all_data["Community"].values
burn = all_data["MTBS Classified dNBR"].values

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(
    3, 2, figsize=(12, 10), sharex=True, sharey=True
)
fig.suptitle("Burn severity histograms by community")
all_counts = []
for i, ax in enumerate((ax1, ax2, ax3, ax4, ax5, ax6)):
    
    arr = burn[comm == (i + 1)]
    
    counts = {}
    for j in range(1, 5):
        label = labels_dnbr[j - 1].replace(" ", "\n", 1)
        counts[label] = np.sum(arr == j)
    
    # Calculate percentages
    pcts = [f"{c / sum(counts.values()):.0%}" for c in counts.values()]
    all_counts.extend(counts.values())

    # Label bars
    container = ax.bar(counts.keys(), counts.values(), color=dnbr_colors)
    ax.bar_label(container, pcts)
    ax.set(title=labels_vegetation[i], yticklabels=[])

    # Rotate labels
    for label in ax.get_xticklabels():
        label.set_rotation(90) 

# Pad ylim so percentage labels fit
ax.set(ylim=(0, max(all_counts) * 1.2))

pass

**Description:** Severe burns are concentrated in oak woodlands > riparian > chaparral. The bands of red pixels in the dNBR images correspond mostly to a mixture of these three communities. Oaks may be driving this, since their distribution closely aligns with the distribution of red pixels.

### Fraction alive

Fraction-alive (FAL or RFAL) measures the ratio of healthy vegetation (green) to total vegetation (green plus brown). Living vegetation on the 2016 FAL plot has a similar distribution to the areas of highest burn severity during the Woolsey Fire, which was one of the observations that lead to this project. FAL data is only available through 2016, which leaves a two-year gap between the last snapshot of vegetation health and the Woolsey Fire, but pre-fire spectral indices like NDWI show a similar pattern in 2018.

In [ ]:
# Plot fraction alive. The FAL plots are all huge because smaller plots have
# large white patches that look to me like a problem with interpolating NaNs.
plot_many(all_data["FAL (2013-2016)"],
          range(2013, 2017),
          fire_bound,
          suptitle="Fraction Alive (FAL)",
          cmap="PiYG",
          vmin=0,
          vmax=1)

#### dFAL

dFAL is the year-on-year difference in fraction alive. I calculated this index by subtracting the FAL of the previous year from the current year, so a positive dFAL means that the fraction alive has increased and a negative dFAL means that fraction alive has decreased.

In [ ]:
# Plot year-on-year difference in fraction alive
plot_many(all_data["dFAL (2013-2016)"],
          ("2013-2014", "2014-2015", "2015-2016"),
          fire_bound,
          suptitle="Change in Fraction Alive (dFAL)",
          cmap="PiYG",
          vmin=-1,
          vmax=1)

#### Dead pixels and the dieback threshold

Foster et al. (2017) use a threshold of 0.5431 to distinguish dead pixels from
live pixels based on a comparison of FAL and a field survey of oaks. The
correlation between the aerial and field surveys is poor and only looked at one
of the six vegetation communities in the study area, so that threshold is
poorly constrained. I've used a threshold of 0.5 below to identify dead pixels.

In [ ]:
# Plot dead pixels
plot_many(all_data["Dead (2013-2016)"],
          range(2013, 2017),
          fire_bound,
          suptitle="Dead Pixels (FAL < 0.5)",
          cmap="Greys",
          cbar=False)

#### Years dead

Using the same threshold as above, the years-dead plot tries to quantify how long pixels have been dead for (that is, how long FAL has been less than 0.5). Years are given relative to 2018, although FAL data for 2017 and 2018 is not available. This calculation has some semantic problems at the other edge, too: Anything classified as dead in 2013 was assigned a value of 5, but those pixels could have been like that for way longer than that.

The idea behind this metric is that vegetation that has been dead longer may be drier and easier to ignite.

In [ ]:
# Set size and style
sns.set(font_scale=1.2, style="white")

# Create custom color map
cmap = ListedColormap(["green", "lightgreen", "yellow", "tan", "brown", "red"])

fig, ax = plt.subplots(figsize=(12, 7))
plot_bands(all_data["Years Dead"],
           ax=ax,
           title="Years Dead (FAL < 0.5)",
           cmap=cmap,
           cbar=None)

fire_bound.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=2)

# Draw legend. This function forces the font size to 13, so the plot canvas
# can only be so big before the legend becomes illegible.
draw_legend(ax.get_images()[0], 
            titles=[str(i) for i in range(6)],
            classes=range(6))

pass

### Climate data for 2013-2016

The SMM stack includes climate data from the PRISM project. The area burned by the Woolsey Fire experienced severe drought from mid-2013 through early 2017. The climate data covers most of this range but does not go all the way through the Woolsey Fire in Nov 2018. Climate data is aggregated to the water year (Oct 1-Sep 30), not the calendar year, and covers the worst part of the 2013-2017 California drought. 

Based on maps from the U.S. Drought Monitor, the Santa Monica Mountains were in:

+ **Exceptional drought** from Jul 2014 to Jan 2017
+ **Extreme drought** from Jan 2014 to Jan 2017
+ **Severe drought** from Jul 2013 to Jan 2017
+ **Moderate drought** from Mar 2012 on

In the plots below, green can generally be understood as being wetter/cooler and yellow as hotter/drier. Colors in each set of plots are scaled to the same range. The highest, driest conditions in the area burned during the Woolsey Fire generally occurred in the northeastern part of the scar.

#### Days with precipitation over 0.1 inchdes

In [ ]:
# Plot number of days with precipiation over 0.1 inches
key = "Days Precipitation (2013-2016)"
xda = all_data[key]

plot_many(xda,
          range(2013, 2017),
          fire_bound,
          suptitle=key,
          cmap="summer_r",
          vmin=xda.min(),
          vmax=xda.max())

#### Maximum vapor pressure deficit

The vapor pressure deficit (VPD) measures how much water would have to added to the air before it would be saturated. High values indicate drier conditions.

In [ ]:
# Plot maximum VPD
key = "Max VPD (2013-2016)"
xda = all_data[key]

plot_many(xda,
          range(2013, 2017),
          fire_bound,
          suptitle=key,
          cmap="summer",
          vmin=xda.min(),
          vmax=xda.max())

#### Minimum temperature

In [ ]:
# Plot minimum temperatue
key = "Minimum Temperature (2013-2016)"
xda = all_data[key]

plot_many(xda,
          range(2013, 2017),
          fire_bound,
          suptitle=key,
          cmap="summer_r",
          vmin=xda.min(),
          vmax=xda.max())

#### Heat days over 95 °F

In [ ]:
# Plot number of days over 95 F
key = "Heat Days Over 95 (2013-2016)"
xda = all_data[key]

plot_many(xda,
          range(2013, 2017),
          fire_bound,
          suptitle=key,
          cmap="summer",
          vmin=xda.min(),
          vmax=xda.max())

#### Cumulative precipitation

In [ ]:
# Plot cumulative precipitation
key = "Cumulative Precipitation (2013-2016)"
xda = all_data[key]

plot_many(xda,
          range(2013, 2017),
          fire_bound,
          suptitle=key,
          cmap="summer_r",
          vmin=xda.min(),
          vmax=xda.max())

### Climate data for 2018

Precipitation might be an important driver for the Woolsey Fire, so I grabbed climate data from PRISM for 2012-2018. **Do not mix and match with the other PRISM data.** As noted above, the PRISM data in the SMM stack is aggreated to the water year, whereas the data downloaded from PRISM is aggregated to the calendar year.

In [ ]:
# Plot 2018 climate data from PRISM
keys = [
    "Precipitation (2018)",
    "Mean Dew Point Temperature (2018)",
    "Maximum Temperature (2018)",
    "Mean Temperature (2018)",
    "Minimum Temperature (2018)",
    "Maximum VPD (2018)",
    "Minimum VPD (2018)"
]

plot_many(prism_2018,
          keys,
          fire_bound,
          suptitle="PRISM Climate Data (2018)",
          cmap="summer_r")

## Previous fires

Most of the Woolsey Fire scar burned at some point between 1927 and the Woolsey Fire in 2018. This section summarizes the recent fire history and shows where previous fires may have impacted the severity of the Woolsey Fire.

In [ ]:
# Set size and style
sns.set(font_scale=1.2, style="white")

# Classify last burned by decade
years = list(range(1930, 2030, 10))
bins = [-np.inf] + years[:-1] + [np.inf]
titles = [f"{y - 10}s" for y in years]

cl_burned = xr.apply_ufunc(np.digitize, all_data["Last Burned"], bins)
cl_burned = cl_burned.astype(float).rio.write_nodata(np.nan)
cl_burned = cl_burned.rio.clip(fire_bound.geometry, drop=False)

# Create a color map with the right number of colors
cmap = cm.get_cmap("Set3")
cmap = ListedColormap([cmap(i / 12) for i in range(12)][:len(years)])

# Create plot comparing recently burned areas to dNBR
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

plot_bands(all_data["MTBS Classified dNBR"],
           ax=ax1,
           cmap=cmap_dnbr,
           vmin=1,
           vmax=5,
           title=(f"{fire_name} - MTBS classified dNBR - {res} m\n"
                   "Dec 15, 2017 - Dec 15, 2018"),
           cbar=False,
           scale=False)

# Plot classified last burned
plot_bands(cl_burned,
           ax=ax2,
           cmap=cmap,
           title=("Recent fires in the vicinity of the Woolsey Fire - NIFC\n"
                  "1927-2017"),
           cbar=None,
           scale=False)

# Draw legend
ep.draw_legend(ax2.get_images()[0], 
               titles=titles,
               classes=range(len(years)))

for ax in (ax1, ax2):
    fire_bound.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1)

pass

**Description:** MTBS dNBR plot (left) and areas of recent wildfires (right). Two areas dark green areas on the dNBR plot--one in the southeast corner and the other on the southwest edge of the fire scar--are difficult to distinguish based on vegetation, topography, or climate from nearby burned areas but do correspond to areas that have burned since 2000.


## Live fuel moisture content

Live fuel moisture content (LFMC) is an estimate of how dry/ignitable potential wildfire fuel is. The LFMC estimates plotted here were calculated for 1 Nov 2018 and were downloaded from a [Google Earth Engine app](https://kkraoj.users.earthengine.app/view/live-fuel-moisture) created using the neutral-network model described by Rao et al. (2020). Lower values indicate drier fuel.

In [ ]:
# Set size and style
sns.set(font_scale=1.2, style="white")

# Plot LFMC data immediately preceding the Woolsey Fire
fig, ax = plt.subplots(1, 1, figsize=(8, 7))
plot_bands(
    all_data["LFMC (2018)"],
    ax=ax,
    title=(f"{fire_name} - Live fuel moisture content - {res} m\n1 Nov 2018"),
    cmap="summer_r"
)
fire_bound.plot(ax=ax, edgecolor="black", facecolor="none", linewidth=1)

pass

**Description:** The LFMC plot shows generally dry conditions immediately preceding the Woolsey Fire. The resolution of the downloaded data looks coarser than the nominal resolution of 30 m and the band of relatively healthy and moist vegetation observed in the FAL and spectral index plots is barely discernible here.

## Histograms

I plotted these histograms to look for parameters where moderate- to high-severity burns could be distinguished from lower-severity burns. Each histogram plots pixel counts for each class on the MTBS classified dNBR dataset for a single parameter. Few pixels were classified as high-severity burns, so the plots use a log scale to make the distribution of each class clear.

The histograms mostly highlight that the different fire severity classes are not clearly separable in any one of the datasets I'm looking at here. However, the distribution of high-severity pixels appears to be skewed relative to the distribution of the other classes on the spectral index, FAL, and folded aspect plots. 

**Note:** These plots are not stacked.

In [ ]:
# Set size and style
sns.set(font_scale=1.2, style="white")

# Plot histograms
keys = [
    "FAL (2013-2016)",
    "LFMC (2018)",
    "NDMI (2018)",
    "NDVI (2018)",
    "NDWI (2018)",
    "SAVI (2018)",
    "Slope",
    "Folded Aspect",
    "Elevation",
    "Last Burned"
]

# Set masks for different burn severities
any_sev = np.where(all_data["MTBS Classified dNBR"] > 1, True, False)
low_sev = np.where(all_data["MTBS Classified dNBR"].values == 2, True, False)
mod_sev = np.where(all_data["MTBS Classified dNBR"].values == 3, True, False)
high_sev = np.where(all_data["MTBS Classified dNBR"].values == 4, True, False)

# Set kwargs for the histogram on each axis
kwargs = {
    "log": True,
}

# Use the same colors for burn severity used by the MTBS plot
colors = ["#CCCCCC", "#7FFFD4", "#FFFF00", "#FF0000"]

# Plot histograms
fig, axes = plt.subplots(len(keys), 1, figsize=(8, 4 * len(keys)))
fig.suptitle("Histograms for moderate and high severity pixels")
for key, ax in zip(keys, axes):
    
    x = all_data[key]
    if len(x.shape) > 2:
        x = x[-1]
    
    for i, x in enumerate([
        np.sort(np.ravel(x.values[any_sev])),
        np.sort(np.ravel(x.values[low_sev])),
        np.sort(np.ravel(x.values[mod_sev])),
        np.sort(np.ravel(x.values[high_sev])),
    ]):
        if not i:
             _, bins, _ = ax.hist(x, bins=20, color=colors[i], **kwargs)
        else:
            ax.hist(x, bins=bins, color=colors[i], **kwargs)
        
        ax.set(xlabel=key, ylabel="Count")
        ax.legend(["all", "low", "mod", "high"])